In [3]:
import librosa2
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import scipy
import pandas as pd
%matplotlib inline
import os
import csv
import natsort
from openpyxl import load_workbook
import scipy.io

SET THE DESIRED OUTPUT FILENAME AND LOCATION

In [12]:
#admin stuff
# filenames must be of type label_h_*_filenam, where * is a certain impulse response legnth
# e.g. '180_h_10_music-hd-0006'
num_MFCC = 20
data_name = f'data6'
IR_lenght = 500
newFileName = f'D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/reverse_Mel_scale/{data_name}_{num_MFCC}_{IR_lenght}ms_R.csv' #don't forget the '.csv'
# newFileLocation = '' #empyt if current folder suffices
readAudioFrom = f'D:/Users/MC/Documents/UNI/MASTER/thesis/AUGMENTED_DATA/{data_name}'
impulseResponseLength = f'h_{IR_lenght}'

In [13]:
#preparing header
# consist of original filename the mean and variance of each Mel frequency bank, the mean and variance of each corrseponding delta and delta-delta
header = 'filename '
for i in range(0, num_MFCC):
    header += f' mfccM{i}'
    header += f' mfccV{i}'
for i in range(0, num_MFCC):
    header += f' deltaM{i}'
    header += f' deltaV{i}'
for i in range(0, num_MFCC):
    header += f' deltaDeltaM{i}'
    header += f' deltaDeltaV{i}'
header += ' label'
header = header.split()
file = open(newFileName, 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [14]:
#reading dataset-header from csv
data = pd.read_csv(newFileName)
data.head()

,filename,mfccM0,mfccV0,mfccM1,mfccV1,mfccM2,mfccV2,mfccM3,mfccV3,mfccM4,...,deltaDeltaV15,deltaDeltaM16,deltaDeltaV16,deltaDeltaM17,deltaDeltaV17,deltaDeltaM18,deltaDeltaV18,deltaDeltaM19,deltaDeltaV19,label


In [15]:
#READING .wav
#read in an audio file, extract the features and write it to a csv file
directions = '0 9 18 27 36 45 54 63 72 81 90 99 108 117 126 135 144 153 162 171 180 189 198 207 216 225 234 243 252 261 270 279 288 297 306 315 324 333 342 351'.split()
for d in directions:
    print(d)
    for filename in os.listdir(f'{readAudioFrom}/{d}'):
        if filename.startswith(f"{d}_{impulseResponseLength}_"):
            #get audio data
            audio_file = f'{readAudioFrom}/{d}/{filename}'
            y,sr = librosa2.load(audio_file, sr=44100)
            
            #get features
            fftWindowSize = 1024                 #~23ms (default:2048 =~46ms @44.1kHz)
            overlapSize = int(fftWindowSize/4)   #ovelap 75%
            numOfMelFilterBanks = num_MFCC             #amount of Mel filterbanks
            mfcc = librosa2.feature.reverse_mfcc(y=y, sr=sr, n_mfcc=numOfMelFilterBanks, n_mels=numOfMelFilterBanks, n_fft=fftWindowSize, hop_length=overlapSize)
            delta = librosa.feature.delta(mfcc)
            deltaDelta = librosa.feature.delta(mfcc, order=2)
            
            #write to csv
            to_append = f'{filename} ' 
            #average psd per filterbank and write to file
            for i in mfcc:
                to_append += f' {np.mean(i)}' 
                to_append += f' {np.var(i)}' 
            #average speed
            for j in delta:
                to_append += f' {np.mean(j)}'
                to_append += f' {np.var(j)}'
            #average acceleration
            for k in deltaDelta:
                to_append += f' {np.mean(k)}'
                to_append += f' {np.var(k)}'
            #write label to file
            to_append += f' {d}'
            file = open(newFileName, 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

0
9
18
27
36
45
54
63
72
81
90
99
108
117
126
135
144
153
162
171
180
189
198
207
216
225
234
243
252
261
270
279
288
297
306
315
324
333
342
351


READ THE NEWLY CREATED FILE AND PRINT THE FIRST COUPLE OF ROWS

In [17]:
#reading dataset from csv
data = pd.read_csv(newFileName)
data.head()

,filename,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,...,deltaDelta34471,deltaDelta34472,deltaDelta34473,deltaDelta34474,deltaDelta34475,deltaDelta34476,deltaDelta34477,deltaDelta34478,deltaDelta34479,label
0,0_h_1000_101-126860-0049.wav,-490.160034,-479.524078,-473.436157,-471.812073,-471.740875,-471.714630,-470.776703,-470.572479,-470.773926,...,0.162921,0.052074,-0.043956,-0.157890,-0.290446,-0.290446,-0.290446,-0.290446,-0.290446,0.0
1,0_h_1000_1031-133220-0062.wav,-473.572205,-458.484772,-453.790527,-452.755463,-451.521332,-452.064880,-453.049866,-450.624725,-447.324799,...,0.083799,0.110981,0.130579,0.062785,-0.005921,-0.005921,-0.005921,-0.005921,-0.005921,0.0
2,0_h_1000_1046-133224-0038.wav,-481.570953,-477.516052,-474.305878,-475.080627,-474.947754,-475.304657,-474.491394,-474.334290,-474.700531,...,0.062722,0.095795,0.077210,0.032369,-0.027117,-0.027117,-0.027117,-0.027117,-0.027117,0.0
3,0_h_1000_1058-131113-0000.wav,-461.966431,-443.989258,-438.321075,-439.984375,-440.390228,-441.697571,-443.083496,-442.517120,-442.449188,...,0.209518,0.278748,0.203279,0.011174,-0.118005,-0.118005,-0.118005,-0.118005,-0.118005,0.0
4,0_h_1000_1061-142358-0026.wav,-470.992706,-454.287476,-448.334167,-447.687103,-448.253479,-448.275604,-448.043793,-448.383636,-448.876892,...,-0.009494,-0.021353,-0.014124,-0.017078,-0.006452,-0.006452,-0.006452,-0.006452,-0.006452,0.0


CHECK WHETHER WHAT THE FILE IS NAMED CORRESPONDS TO WHAT THE ENTRIES OF THE FILE ARE <br>
this uses a simple string compare and is not foolproof

In [6]:
#check for discrepancy between filename and file contents
# a possible error could be that the file with name 'data6_1000ms.csv' contains an entry '0_h_50_music-fma-0006.wav'
for filename in os.listdir('D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/reverse_Mel_scale'):
    split_filename_list = filename.split("_")
    filename_list_end = split_filename_list[1].split("ms")
    filter = f'h_{filename_list_end[0]}_'
    dataFileEntry = pd.read_csv(os.path.join('D:/Users/MC/Documents/UNI/MASTER/thesis/MFCC_FEATURES2/reverse_Mel_scale',filename))
    entryName = dataFileEntry['filename'].iloc[0]
    if filter not in entryName:
        print('\nERROR')
        print(f'filename is "{filename}" yet contains {entryName}')
print('all files checked')


ERROR
filename is "data5_unseen_1000ms_R.csv" yet contains 0_h_1000_1195-130164-0023.wav

ERROR
filename is "data5_unseen_100ms_R.csv" yet contains 0_h_100_1195-130164-0023.wav

ERROR
filename is "data5_unseen_10ms_R.csv" yet contains 0_h_10_1195-130164-0023.wav

ERROR
filename is "data5_unseen_500ms_R.csv" yet contains 0_h_500_1195-130164-0023.wav

ERROR
filename is "data5_unseen_50ms_R.csv" yet contains 0_h_50_1195-130164-0023.wav

ERROR
filename is "data6_unseen_1000ms_R.csv" yet contains 0_h_1000_music-fma-0003.wav

ERROR
filename is "data6_unseen_100ms_R.csv" yet contains 0_h_100_music-fma-0003.wav

ERROR
filename is "data6_unseen_10ms_R.csv" yet contains 0_h_10_music-fma-0003.wav

ERROR
filename is "data6_unseen_500ms_R.csv" yet contains 0_h_500_music-fma-0003.wav

ERROR
filename is "data6_unseen_50ms_R.csv" yet contains 0_h_50_music-fma-0003.wav

ERROR
filename is "data6_validation_1000ms_R.csv" yet contains 0_h_1000_music-fma-0000.wav

ERROR
filename is "data6_validation_100ms